# Final results notebook

In [1]:
from datasets import load_dataset
import numpy as np
import pandas as pd
import random

/Users/sarrabenyahia/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Data

In [2]:
train_dataset = load_dataset("snli", split='train')
test_dataset = load_dataset("snli", split='test')
validation_dataset = load_dataset("snli", split='validation')

Found cached dataset snli (/Users/sarrabenyahia/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)
Found cached dataset snli (/Users/sarrabenyahia/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)
Found cached dataset snli (/Users/sarrabenyahia/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)


## Pre-processing

In [3]:
from data_utils import PreProcessor, Filter
filter = Filter()
preprocessor = PreProcessor()

train_dataset_filtered = filter.transform(train_dataset)
test_dataset_filtered = filter.transform(test_dataset)
validation_dataset_filtered = filter.transform(validation_dataset)

train_dataset_processed, labels_train = preprocessor.fit_transform(train_dataset_filtered)
test_dataset_processed, labels_test = preprocessor.transform(test_dataset_filtered)
validation_dataset_processed, labels_val = preprocessor.transform(validation_dataset_filtered)

Loading cached processed dataset at /Users/sarrabenyahia/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-89060f0ac6d2a87b.arrow
Loading cached processed dataset at /Users/sarrabenyahia/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-d880ac165b0b8f91.arrow
Loading cached processed dataset at /Users/sarrabenyahia/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-f62e40a99b36d61b.arrow


## Model

#### Train

In [4]:
from model import RobertaPredictor

transformer_trainer = RobertaPredictor('cross-encoder/nli-roberta-base')
train_data_sample = random.sample(train_dataset_processed, k=1000)
transformer_trainer.fit(train_data_sample, validation_dataset_processed)


/Users/sarrabenyahia/Library/Python/3.9/lib/python/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 125
  Number of trainable parameters = 124647939


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 125/125 [03:45<00:00,  1.76s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 125/125 [03:45<00:00,  1.80s/it]

{'train_runtime': 225.6197, 'train_samples_per_second': 4.432, 'train_steps_per_second': 0.554, 'train_loss': 0.6458991088867188, 'epoch': 1.0}


#### Predict

In [5]:
# Call the predict method of the TransformerTrainer object on your test data
predicted_labels, accuracy = transformer_trainer.predict(test_dataset_processed)
print(f'The accuracy of the model is {accuracy}')

***** Running Prediction *****
  Num examples = 9824
  Batch size = 32
100%|██████████| 307/307 [04:50<00:00,  1.06it/s]

The accuracy of the model is 0.9022801302931596


#### Predicting on a bigger sample

In [7]:
from model import RobertaPredictor

transformer_trainer = RobertaPredictor('cross-encoder/nli-roberta-base')
train_data_sample = random.sample(train_dataset_processed, k=10000)
transformer_trainer.fit(train_data_sample, validation_dataset_processed)

loading configuration file config.json from cache at /Users/sarrabenyahia/.cache/huggingface/hub/models--cross-encoder--nli-roberta-base/snapshots/1c9dadfb1d7bcaac49176fd3a5de914f6ae2bd42/config.json
Model config RobertaConfig {
  "_name_or_path": "cross-encoder/nli-roberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "contradiction",
    "1": "entailment",
    "2": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "contradiction": 0,
    "entailment": 1,
    "neutral": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolu

{'loss': 0.4862, 'learning_rate': 3.0002400192015362e-05, 'epoch': 0.4}



 40%|████      | 500/1250 [25:17<32:48,  2.62s/it]Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json


{'eval_loss': 0.3121122717857361, 'eval_accuracy': 0.8962609225767121, 'eval_runtime': 450.6324, 'eval_samples_per_second': 21.84, 'eval_steps_per_second': 0.683, 'epoch': 0.4}


Model weights saved in ./results/checkpoint-500/pytorch_model.bin
 80%|████████  | 1000/1250 [43:05<09:04,  2.18s/it]   ***** Running Evaluation *****
  Num examples = 9842
  Batch size = 32


{'loss': 0.3774, 'learning_rate': 1.0000800064005119e-05, 'epoch': 0.8}


                                                   
 80%|████████  | 1000/1250 [50:52<09:04,  2.18s/it]Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json


{'eval_loss': 0.3315350413322449, 'eval_accuracy': 0.90367811420443, 'eval_runtime': 466.5435, 'eval_samples_per_second': 21.096, 'eval_steps_per_second': 0.66, 'epoch': 0.8}


Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
100%|██████████| 1250/1250 [1:00:22<00:00,  2.25s/it] 

Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/checkpoint-500 (score: 0.3121122717857361).
100%|██████████| 1250/1250 [1:00:22<00:00,  2.90s/it]

{'train_runtime': 3622.503, 'train_samples_per_second': 2.761, 'train_steps_per_second': 0.345, 'train_loss': 0.41317313232421876, 'epoch': 1.0}


In [8]:
# Call the predict method of the TransformerTrainer object on your test data
predicted_labels, accuracy = transformer_trainer.predict(test_dataset_processed)
print(f'The accuracy of the model is {accuracy}')

***** Running Prediction *****
  Num examples = 9824
  Batch size = 32
100%|██████████| 307/307 [05:38<00:00,  1.10s/it]

The accuracy of the model is 0.8927117263843648
